<div class="frontmatter text-center">
<h1>Geospatial Data Science</h1>
<h2>Exercise 4: Spatial Weights</h2>
<h3>IT University of Copenhagen, Spring 2023</h3>
<h3>Instructor: Jan Leonard Schelhaas</h3>
</div>

This notebook was adapted from:
* A course on geographic data science: https://darribas.org/gds_course/content/bE/diy_E.html

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import contextily as cx
from pysal.lib import examples
from pysal.lib import weights
import matplotlib.pyplot as plt

## Task I: Contiguity Weigths & NYC Tracts

In this task we will explore **contiguity** weights. To do it, we will load Census tracts for New York City. Census tracts are the geography the US Census Burearu uses for areas around 4,000 people. We will use a dataset prepared as part of the [PySAL examples](https://pysal.org/libpysal/tutorial.html?highlight=examples#example-datasets). Geographically, this is a set of polygons that cover all the area of the city of New York. 

We can get information on the dataset and its variables directly from pysal.lib.examples (or [here](https://geodacenter.github.io/data-and-lab/NYC_Tract_ACS2008_12/)).

In [ ]:
examples.explain("NYC Socio-Demographics")

To check out the location of the files that make up the dataset, we can load it with `load_example` and inspect with `get_file_list`:

In [ ]:
# Load example (this automatically downloads if not available)
nyc_data = examples.load_example("NYC Socio-Demographics")
# Print the paths to all the files in the dataset
nyc_data.get_file_list()

And now let's read the shapefile:

In [ ]:
nyc = gpd.read_file(nyc_data.get_path("NYC_Tract_ACS2008_12.shp"))
nyc.plot(figsize=(9, 9));

### Task 1a: Create a contiguity matrix using the queen criterion.

<div class="alert alert-block alert-info">
<b>Note:</b> PySal will warn us that using contigency for weights results in 7 disconnected components and 3 islands. This is not a problem for our task, but if the data were used in a real analysis, we might want to consider using a different method for creating spatial weights!

You might also get warning about islands and disconnected components later in the analysis. These should not pose a problem, but think about how you could solve this?
</div>

### Task 1b: Central Park Neighbours 
* Let's focus on Central Park. Find the corresponding census tract in our nyc dataset. [This map](https://hub.arcgis.com/datasets/DCP::nyc-census-tracts-for-2010-us-census?geometry=-76.267%2C40.341%2C-71.688%2C41.069) can help.
* How many neighbours does Central Park have?
* Make a plot of Central Park and the surrounding area (zoomed in), highlighting all neighbours of Central Park - creating a similar plot as in the lecture (cf. *"Queen neighbors of 708518"*)

In [ ]:
# Add your code here

### Task 1c: Block weights by borough
* Using the 5 boroughs of NYC, create a block spatial weights matrix where every tract is connected to other tracts in the same borough.

In [ ]:
# Add your code here

### Task 1d: Mean number of neighbours for contiguity and block matrices
* Compare the number of neighbours for the contiguity matrix and the block spatial weights matrix. 
* Which criterion gives you which number of neighbours - and why the big difference?*

In [ ]:
# Add your code here

## Task 2: Distance Weights & Japanese cities

In this task, you will be generating spatial weights matrices based on **distance**. We will use a [dataset](http://www.oecd.org/cfe/regionaldevelopment/functional-urban-areas.htm) of Japanese urban areas provided by [OECD](http://www.oecd.org). Let's get it ready for you to work on it directly.

The data are available over the web on the following address and we can read straight into a GeoDataFrame:

In [ ]:
jp = gpd.read_file("http://www.oecd.org/cfe/regionaldevelopment/Japan.zip")

If this does not work, you can find the data in the data folder ('Japan.shp').

In [ ]:
# inspect the data set
jp.head()

If we make a quick plot, we can see these are polygons covering the part of the Japanese geography that is considered urban:

In [ ]:
ax = jp.plot(color="red", alpha=0.5, figsize=(9, 9))
cx.add_basemap(ax, crs=jp.crs)

For this example, we need two transformations: 
- lon/lat coordinates to a geographical projection
- polygons to points. 

To calculate distances effectively, we need to ensure the coordinates of our geographic data are expressed in metres (or a similar measurement unit). The original dataset is expressed in lon/lat degrees:

In [ ]:
jp.crs

We can use the Japan Plane Rectangular CS XVII system ([`EPSG:2459`](http://epsg.io/2459)), which is expressed in metres:

In [ ]:
jp = jp.to_crs(epsg=2459)

So the resulting table is in metres:

In [ ]:
jp.crs

Distances are easier to calculate between points than between polygons. Hence, we will **convert the urban areas into their centroids**:

In [ ]:
jp_c = jp.copy()
jp_c.geometry = jp.geometry.centroid

In [ ]:
jp_c.head()

So the result is a seet of points expressed in metres:

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (10,10))
jp.plot(ax = ax, color ="black")
jp_c.plot(ax = ax, markersize = 4, color = "red")
cx.add_basemap(ax, crs=jp.crs)
plt.show()

### Task 2a
* Create two spatial weights matrices based on **distance**: one with the five nearest neighbours; one with a 100km distance band. 
* Remember to consider the correct distance unit in the dataset!
* Compare the mean number of neighbours for both spatial weight matrices. 
* **What are the main differences you can spot? In which cases do you think one criterion would be preferable over the other?**

In [ ]:
# Your code for task 2a here

### Task 2b: Tokyo Plotting Challenge (optional)

Focus on Tokyo and the 100km spatial weights matrix generated above. Make a plot where you visualize Tokyo, the 100km distance band, and highlight all neighbouring cities that lie within the distance band.

An optional recipe to follow:

0. Find Tokyo in our jp dataset
1. Generate a [buffer](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html) of 100km around the Tokyo centroid.
2. Start the plot with the Tokyo urban area polygon (from `jp_cities`)
3. Plots its neighbours (within the buffer)
4. Plot the neighbours' centroids (points)
5. Plot the buffer on top
6. Add a basemap

Your figure should look similar to:

In [ ]:
from IPython.display import Image
Image("https://darribas.org/gds_course/_images/diy_E_24_1.png")

In [ ]:
# Your code for task 2b here